<a href="https://colab.research.google.com/github/ivaben/DS-Unit-2-Linear-Models/blob/master/I_BERNSTEIN__regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])


In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [627]:
# Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [628]:
df.shape

(421, 59)

In [629]:
# Convert to datetime and look at the date range
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df['Date'].describe()

count                     421
unique                    169
top       2016-08-30 00:00:00
freq                       29
first     2011-05-16 00:00:00
last      2026-04-25 00:00:00
Name: Date, dtype: object

In [630]:
df.dtypes

Burrito                   object
Date              datetime64[ns]
Yelp                     float64
Google                   float64
Chips                     object
Cost                     float64
Hunger                   float64
Mass (g)                 float64
Density (g/mL)           float64
Length                   float64
Circum                   float64
Volume                   float64
Tortilla                 float64
Temp                     float64
Meat                     float64
Fillings                 float64
Meat:filling             float64
Uniformity               float64
Salsa                    float64
Synergy                  float64
Wrap                     float64
Unreliable                object
NonSD                     object
Beef                      object
Pico                      object
Guac                      object
Cheese                    object
Fries                     object
Sour cream                object
Pork                      object
Chicken   

In [0]:
train = df[(df['Date'] <= "12/31/2016")]


In [632]:
train.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,2016-01-24,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,2016-01-24,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,2016-01-24,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,2016-01-27,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [0]:
validate = df[(df['Date'] >= "1/1/2017") & (df['Date'] < "1/1/2018")]


In [634]:
validate.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
301,California,2017-01-04,NaN,NaN,NaN,6.60,NaN,NaN,NaN,23.0,20.5,0.77,4.0,4.5,4.0,3.5,3.5,5.0,1.5,3.50,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
302,Other,2017-01-04,NaN,NaN,NaN,6.60,NaN,NaN,NaN,20.5,21.5,0.75,4.0,2.0,NaN,4.0,NaN,4.6,4.2,3.75,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
303,Other,2017-01-07,NaN,NaN,NaN,8.50,3.9,NaN,NaN,21.0,21.0,0.74,3.0,4.5,4.1,3.0,3.7,4.0,4.3,4.20,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
304,Other,2017-01-07,NaN,NaN,NaN,7.90,4.0,NaN,NaN,20.5,21.0,0.72,3.5,4.0,4.0,3.0,4.0,4.5,4.0,3.80,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
305,Other,2017-01-10,NaN,NaN,NaN,4.99,3.5,NaN,NaN,18.5,22.5,0.75,2.5,4.5,3.0,2.5,3.0,3.0,2.0,2.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [635]:
test = df[(df['Date'] >= "1/1/2018")]
test.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
77,California,2026-04-25,NaN,NaN,NaN,8.00,4.0,NaN,NaN,21.59,NaN,NaN,4.5,5.0,5.0,5.0,4.5,5.0,3.0,5.0,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
386,California,2018-01-02,NaN,NaN,NaN,7.25,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,5.0,5.0,3.0,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
387,Other,2018-01-09,4.5,3.8,NaN,4.19,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,2.0,2.0,4.0,1.0,4.0,3.0,4.0,x,NaN,NaN,NaN,NaN,x,NaN,x,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,False
388,California,2018-01-12,3.5,4.3,NaN,7.00,5.0,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
389,Other,2018-01-12,NaN,NaN,NaN,8.50,4.0,NaN,NaN,21.00,23.5,0.92,4.0,4.0,3.0,3.5,1.0,2.0,3.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [636]:
df.isna().sum() / len(df)*100

Burrito             0.000000
Date                0.000000
Yelp               79.334917
Google             79.334917
Chips              93.824228
Cost                1.662708
Hunger              0.712589
Mass (g)           94.774347
Density (g/mL)     94.774347
Length             32.779097
Circum             33.254157
Volume             33.254157
Tortilla            0.000000
Temp                4.750594
Meat                3.325416
Fillings            0.712589
Meat:filling        2.137767
Uniformity          0.475059
Salsa               5.938242
Synergy             0.475059
Wrap                0.712589
Unreliable         92.161520
NonSD              98.337292
Beef               57.482185
Pico               62.470309
Guac               63.420428
Cheese             62.232779
Fries              69.833729
Sour cream         78.147268
Pork               87.885986
Chicken            95.011876
Shrimp             95.011876
Fish               98.574822
Rice               91.448931
Beans         

In [0]:
# droping columns with more than 95% NaN
df = df.drop(columns=['NonSD', 'Chicken', 'Shrimp', 'Fish', 'Tomato', 'Bell peper', 
                      'Carrots', 'Cabbage', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 
                      'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso', 
                      'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini'])

In [0]:
df.fillna(0, inplace=True)

In [639]:
df.isna().sum()

Burrito           0
Date              0
Yelp              0
Google            0
Chips             0
Cost              0
Hunger            0
Mass (g)          0
Density (g/mL)    0
Length            0
Circum            0
Volume            0
Tortilla          0
Temp              0
Meat              0
Fillings          0
Meat:filling      0
Uniformity        0
Salsa             0
Synergy           0
Wrap              0
Unreliable        0
Beef              0
Pico              0
Guac              0
Cheese            0
Fries             0
Sour cream        0
Pork              0
Rice              0
Beans             0
Lettuce           0
Sauce             0
Great             0
dtype: int64

In [640]:
train.shape, val.shape, test.shape

((298, 59), (85, 59), (38, 59))

In [641]:
df['Length'].dropna().mean()

13.469881235154393

In [642]:
# Begin with baselines for classification.
train.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,2016-01-24,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,2016-01-24,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,2016-01-24,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,2016-01-27,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [643]:
target = 'Great'
y_train = train[target]
y_train.value_counts(normalize=True)
print(f'The baseline for {target} is {baseline:,.02%}.')

The baseline for Great is 50.00%.


In [644]:
target = 'Great'
y_train = train[target]
y_train.value_counts(normalize=True)
y_validate = validate[target]
y_validate.value_counts(normalize=True)

False    0.552941
True     0.447059
Name: Great, dtype: float64

In [645]:
majority = y_train.mode()[0]
y_pred = [majority] * len(y_train)
len(y_pred)
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_pred)

0.5906040268456376

In [0]:
# Use scikit-learn for logistic regression.

In [647]:
df.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries', 'Sour cream', 'Pork', 'Rice', 'Beans', 'Lettuce', 'Sauce',
       'Great'],
      dtype='object')

In [0]:

features = ['Burrito', 'Cost', 'Hunger', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries']

target = 'Great'
X_train = train[features]
y_train = train[target]
X_validate = val[features]
y_validate = val[target]

In [649]:
X_train.shape, y_train.shape, X_validate.shape, y_validate.shape


((298, 20), (298,), (85, 20), (85,))

In [0]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_validate_encoded = encoder.transform(X_validate)

In [651]:
X_train_encoded.head()

,Burrito_California,Burrito_Carnitas,Burrito_Asada,Burrito_Other,Burrito_Surf & Turf,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef_x,Beef_nan,Beef_X,Pico_x,Pico_nan,Pico_X,Guac_x,Guac_nan,Guac_X,Cheese_x,Cheese_nan,Cheese_X,Fries_x,Fries_nan,Fries_X
0,1,0,0,0,0,6.49,3.0,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
1,1,0,0,0,0,5.45,3.5,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
2,0,1,0,0,0,4.85,1.5,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0
3,0,0,1,0,0,5.25,2.0,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
4,1,0,0,0,0,6.59,4.0,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0


In [652]:
X_validate_encoded.head()

,Burrito_California,Burrito_Carnitas,Burrito_Asada,Burrito_Other,Burrito_Surf & Turf,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Beef_x,Beef_nan,Beef_X,Pico_x,Pico_nan,Pico_X,Guac_x,Guac_nan,Guac_X,Cheese_x,Cheese_nan,Cheese_X,Fries_x,Fries_nan,Fries_X
301,1,0,0,0,0,6.60,NaN,23.0,20.5,0.77,4.0,4.5,4.0,3.5,3.5,5.0,1.5,3.50,4.5,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
302,0,0,0,1,0,6.60,NaN,20.5,21.5,0.75,4.0,2.0,NaN,4.0,NaN,4.6,4.2,3.75,5.0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
303,0,0,0,1,0,8.50,3.9,21.0,21.0,0.74,3.0,4.5,4.1,3.0,3.7,4.0,4.3,4.20,5.0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
304,0,0,0,1,0,7.90,4.0,20.5,21.0,0.72,3.5,4.0,4.0,3.0,4.0,4.5,4.0,3.80,4.8,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
305,0,0,0,1,0,4.99,3.5,18.5,22.5,0.75,2.5,4.5,3.0,2.5,3.0,3.0,2.0,2.00,4.0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0


In [0]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_validate_imputed = imputer.transform(X_validate_encoded)

In [654]:
X_train_imputed[:2]

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         6.49      ,  3.        , 19.82988571, 22.04224138,  0.77091954,
         3.        ,  5.        ,  3.        ,  3.5       ,  4.        ,
         4.        ,  4.        ,  4.        ,  4.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         5.45      ,  3.5       , 19.82988571, 22.04224138,  0.77091954,
         2.        ,  3.5       ,  2.5       ,  2.5       ,  2.        ,
         4.        ,  3.5       ,  2.5       ,  5.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ]])

In [655]:
X_validate_imputed[:2]

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         6.6       ,  3.4452862 , 23.        , 20.5       ,  0.77      ,
         4.        ,  4.5       ,  4.        ,  3.5       ,  3.5       ,
         5.        ,  1.5       ,  3.5       ,  4.5       ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         6.6       ,  3.4452862 , 20.5       , 21.5       ,  0.75      ,
         4.        ,  2.        ,  3.55121528,  4.        ,  3.52886986,
         4.6       ,  4.2       ,  3.75      ,  5.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ]])

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_validate_scaled = scaler.transform(X_validate_imputed)

In [657]:
X_train_scaled[:2]

array([[ 1.23508045, -0.22202652, -0.36480111, -0.76492247, -0.27534029,
        -0.33980523, -0.52430658,  0.        ,  0.        ,  0.        ,
        -0.59316238,  1.34069563, -0.64599247, -0.02244697,  0.4582244 ,
         0.55747757,  0.72124523,  0.50099338,  0.03868647,  1.13679712,
        -0.87966444, -0.38230073,  1.26146912, -1.04111291, -0.32203059,
         1.39660125, -1.06952555, -0.38230073,  1.20946679, -1.        ,
        -0.32203059,  1.43950163, -1.226459  , -0.28232985],
       [ 1.23508045, -0.22202652, -0.36480111, -0.76492247, -0.27534029,
        -1.20857147,  0.0644233 ,  0.        ,  0.        ,  0.        ,
        -1.84902306, -0.21386677, -1.2319636 , -1.20240271, -1.48698931,
         0.55747757,  0.18727406, -1.13340089,  0.89967866,  1.13679712,
        -0.87966444, -0.38230073,  1.26146912, -1.04111291, -0.32203059,
         1.39660125, -1.06952555, -0.38230073,  1.20946679, -1.        ,
        -0.32203059,  1.43950163, -1.226459  , -0.28232985]])

In [658]:
X_validate_scaled[:2]


array([[ 1.23508045e+00, -2.22026518e-01, -3.64801107e-01,
        -7.64922469e-01, -2.75340288e-01, -2.47916498e-01,
         5.22897175e-16,  1.99332912e+00, -1.20123103e+00,
        -8.75592328e-03,  6.62698293e-01,  8.22508165e-01,
         5.25949777e-01, -2.24469694e-02, -2.80790267e-02,
         1.48037108e+00, -1.94861062e+00, -4.38047113e-02,
         4.69182568e-01, -8.79664438e-01,  1.13679712e+00,
        -3.82300727e-01, -7.92726498e-01,  9.60510617e-01,
        -3.22030594e-01, -7.16023991e-01,  9.34994030e-01,
        -3.82300727e-01, -8.26810631e-01,  1.00000000e+00,
        -3.22030594e-01, -6.94684867e-01,  8.15355425e-01,
        -2.82329851e-01],
       [-8.09663853e-01, -2.22026518e-01, -3.64801107e-01,
         1.30732204e+00, -2.75340288e-01, -2.47916498e-01,
         5.22897175e-16,  4.21359674e-01, -4.22344502e-01,
        -1.99197255e-01,  6.62698293e-01, -1.76842918e+00,
        -5.20446908e-16,  5.67530899e-01, -4.31924210e-16,
         1.11121367e+00,  9.34

**FIRST** **ATTEMPT**

In [659]:
from sklearn.linear_model import LogisticRegressionCV

model = LogisticRegressionCV(cv=5, random_state=42).fit(X_train_scaled, y_train)


print('Validation Accuracy is', model.score(X_validate_scaled, y_validate)*100)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Validation Accuracy is 75.29411764705883


In [660]:
model.coef_[0], X_train_encoded.columns

(array([ 0.694999  ,  0.02756307, -0.19479687, -0.6017599 ,  0.02883032,
         0.37738598,  0.0954903 ,  0.15755312, -0.00797189,  0.33991356,
         0.90249184,  1.01135302,  1.88691979,  1.32812765,  1.7429063 ,
        -0.05337895,  0.34023395,  2.3031267 ,  0.28459773, -0.03317859,
         0.24329946, -0.31240432, -0.27392228,  0.33196442, -0.11138765,
         0.29961068, -0.34291045,  0.08769128, -0.1439833 ,  0.21549201,
        -0.12693761, -0.19991115,  0.28896591, -0.18300429]),
 Index(['Burrito_California', 'Burrito_Carnitas', 'Burrito_Asada',
        'Burrito_Other', 'Burrito_Surf & Turf', 'Cost', 'Hunger', 'Length',
        'Circum', 'Volume', 'Tortilla', 'Temp', 'Meat', 'Fillings',
        'Meat:filling', 'Uniformity', 'Salsa', 'Synergy', 'Wrap', 'Beef_x',
        'Beef_nan', 'Beef_X', 'Pico_x', 'Pico_nan', 'Pico_X', 'Guac_x',
        'Guac_nan', 'Guac_X', 'Cheese_x', 'Cheese_nan', 'Cheese_X', 'Fries_x',
        'Fries_nan', 'Fries_X'],
       dtype='object'))

**ANOTHER** **ATTEMPT**

In [661]:
df.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries', 'Sour cream', 'Pork', 'Rice', 'Beans', 'Lettuce', 'Sauce',
       'Great'],
      dtype='object')

In [0]:
features_1 = ['Burrito', 'Cost', 'Hunger', 'Tortilla', 'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap',]

target = 'Great'

X_train_1 = train[features_1]
y_train_1 = train[target]
X_validate_1 = validate[features_1]
y_validate_1 = validate[target]

In [663]:
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded_1= encoder.fit_transform(X_train_1)
X_validate_encoded_1 = encoder.transform(X_validate_1)

X_train_encoded_1.shape, X_validate_encoded_1.shape

((298, 16), (85, 16))

In [664]:
imputer = SimpleImputer()

X_train_imputed_1 = imputer.fit_transform(X_train_encoded_1)
X_validate_imputed_1 = imputer.transform(X_validate_encoded_1)

print(X_train_imputed_1.shape, X_validate_imputed_1.shape)
X_train_imputed_1[:1]

(298, 16) (85, 16)


array([[1.  , 0.  , 0.  , 0.  , 0.  , 6.49, 3.  , 3.  , 5.  , 3.  , 3.5 ,
        4.  , 4.  , 4.  , 4.  , 4.  ]])

In [665]:
scaler = StandardScaler()

X_train_scaled_1 = scaler.fit_transform(X_train_imputed_1)
X_validate_scaled_1 = scaler.transform(X_validate_imputed_1)

print(X_train_scaled_1.shape, X_validate_scaled_1.shape)
X_train_imputed_1[:1]

(298, 16) (85, 16)


array([[1.  , 0.  , 0.  , 0.  , 0.  , 6.49, 3.  , 3.  , 5.  , 3.  , 3.5 ,
        4.  , 4.  , 4.  , 4.  , 4.  ]])

In [666]:
model_1 = LogisticRegressionCV(cv=5, random_state=42)

model_1.fit(X_train_scaled_1, y_train_1)
print('Validation accuracy # 2 is:', model_1.score(X_validate_scaled_1, y_validate_1) *100)


Validation accuracy # 2 is: 87.05882352941177


**ANOTHER** **ATTEMPT**

In [0]:
features_2 = ['Burrito', 'Cost', 'Hunger', 'Tortilla', 'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity',
       'Synergy', 'Wrap',]

target = 'Great'

X_train_2 = train[features_2]
y_train_2 = train[target]
X_validate_2 = validate[features_2]
y_validate_2 = validate[target]

In [668]:
encoder = ce.OneHotEncoder(use_cat_names=True)

X_train_encoded_2= encoder.fit_transform(X_train_2)
X_validate_encoded_2 = encoder.transform(X_validate_2)

X_train_encoded_2.shape, X_validate_encoded_2.shape

((298, 15), (85, 15))

In [669]:
imputer = SimpleImputer()

X_train_imputed_2 = imputer.fit_transform(X_train_encoded_2)
X_validate_imputed_2 = imputer.transform(X_validate_encoded_2)

print(X_train_imputed_2.shape, X_validate_imputed_2.shape)
X_train_imputed_2[:1]

(298, 15) (85, 15)


array([[1.  , 0.  , 0.  , 0.  , 0.  , 6.49, 3.  , 3.  , 5.  , 3.  , 3.5 ,
        4.  , 4.  , 4.  , 4.  ]])

In [670]:
scaler = StandardScaler()

X_train_scaled_2 = scaler.fit_transform(X_train_imputed_2)
X_validate_scaled_2 = scaler.transform(X_validate_imputed_2)

print(X_train_scaled_2.shape, X_validate_scaled_2.shape)
X_train_imputed_2[:1]

(298, 15) (85, 15)


array([[1.  , 0.  , 0.  , 0.  , 0.  , 6.49, 3.  , 3.  , 5.  , 3.  , 3.5 ,
        4.  , 4.  , 4.  , 4.  ]])

In [671]:
model_2 = LogisticRegressionCV(cv=5, random_state=42)

model_2.fit(X_train_scaled_2, y_train_2)
print('Validation accuracy # 3 is:', model_2.score(X_validate_scaled_2, y_validate_2) *100)


Validation accuracy # 3 is: 84.70588235294117


In [672]:
# Get your model's test accuracy. (One time, at the end.)

X_test = test[features_2]
X_test_encoded = encoder.transform(X_test)
X_test_imputed = imputer.transform(X_test_encoded)
X_test_scaled = scaler.transform(X_test_imputed)
model_2.predict(X_test_scaled)


array([ True,  True, False,  True, False, False,  True,  True, False,
        True,  True, False,  True, False,  True,  True,  True, False,
       False, False, False,  True, False,  True,  True, False, False,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True,  True])